In [2]:
import binascii

def textTobin(st):
    b = bytearray()
    b.extend(map(ord, st))
    return bin(int(binascii.hexlify(b),16))[2:].zfill(16)

import re
def isReadable(st): 
    return bool(re.search('^[a-zA-Z0-9\., \'\"\-_\:\(\)]+$', st))

def isValid(st):
    if str("\\x") in str(st):
        return False
    else:
        return True
    
    
def bxor(a1,a2):
    y = int(a1,2) ^ int(a2,2)
    return ('{0:b}'.format(y)).zfill(16)

def hexTobin(a):
    return bin(int(a, 16))[2:]



In [3]:
import sys
from BitVector import * #(A)


PassPhrase = "Person of Interest is an American science fiction crime drama" #(C)
BLOCKSIZE = 16 #(D)
numbytes = BLOCKSIZE // 8 #(E)

# Reduce the passphrase to a bit array of size BLOCKSIZE:
bv_iv = BitVector(bitlist = [0]*BLOCKSIZE) #(F)
print(bv_iv)
#print(len(PassPhrase) // numbytes)

for i in range(0,len(PassPhrase) // numbytes): #(G)
    textstr = PassPhrase[i*numbytes:(i+1)*numbytes] #(H)
    #print("tx",textTobin(textstr))
    bv_iv ^= BitVector( textstring = textstr ) #(I)
    #print("bv",bv_iv)
    
# Get key from user:
key = None
if sys.version_info[0] == 3: #(J)
    key = input("\nEnter key: ") #(K)
else:
    key = raw_input("\nEnter key: ") #(L)
key = key.strip() #(M)


# Reduce the key to a bit array of size BLOCKSIZE:
key_bv = BitVector(bitlist = [0]*BLOCKSIZE) #(N)
for i in range(0,len(key) // numbytes): #(O)
    keyblock = key[i*numbytes:(i+1)*numbytes] #(P)
    key_bv ^= BitVector( textstring = keyblock ) #(Q)z

print(key_bv)   
n = int(str(key_bv), 2)
print(binascii.unhexlify('0%x' % n))

    
# Create a bitvector for storing the ciphertext bit array:
msg_encrypted_bv = BitVector( size = 0 ) #(R)

# Carry out differential XORing of bit blocks and encryption:
previous_block = bv_iv #(S)
bv = BitVector( filename = "inText2.txt" ) #(T)
while (bv.more_to_read): #(U)
    bv_read = bv.read_bits_from_file(BLOCKSIZE) #(V)
    if len(bv_read) < BLOCKSIZE: #(W)
        bv_read += BitVector(size = (BLOCKSIZE - len(bv_read))) #(X)
    bv_read ^= key_bv #(Y)
    bv_read ^= previous_block #(Z)
    previous_block = bv_read.deep_copy() #(a)
    msg_encrypted_bv += bv_read #(b)

    
# Convert the encrypted bitvector into a hex string:
outputhex = msg_encrypted_bv.get_hex_string_from_bitvector() #(c)

# Write ciphertext bitvector to the output file:
FILEOUT = open("inDecryptText2.txt", 'w') #(d)
FILEOUT.write(outputhex) #(e)
FILEOUT.close()


0000000000000000

Enter key: waqe
0000011000000100
b'\x06\x04'


In [4]:
import sys

PassPhrase = "Person of Interest is an American science fiction crime drama" 
BLOCKSIZE = 16


numbytes = BLOCKSIZE // 8 
#print(numbytes)

binpass = [0]*BLOCKSIZE
binpass = ''.join(str(i) for i in binpass)

#print(binpass)


for i in range(0,len(PassPhrase) // numbytes): 
    textstr = PassPhrase[i*numbytes:(i+1)*numbytes] 
    binstr = textTobin(textstr)
    binpass = bxor(binpass,binstr)
    
    

with open("inDecryptText2.txt", 'r') as file:
    content = hexTobin(file.read())
    tempz = [0]*(8-len(content)%8)
    tempz = ''.join(str(i) for i in tempz)
    content = tempz + content
    #print(content)


#Get key from user:
# key = None
# if sys.version_info[0] == 3: #(L)
#     key = input("\nEnter key: ") #(M)
# else:
#     key = raw_input("\nEnter key: ") #(N)
# key = key.strip() #(O)
# #print(key)

import itertools
lst = list(itertools.product([0, 1], repeat=16))
for m in range(len(lst)):
    #print(m)
    ls = list(lst[m])
    st = ''.join(str(j) for j in ls)
    
#     n = int(st, 2)
#     tempkey = n.to_bytes((n.bit_length() + 7) // 8, 'big')
#     if isValid(tempkey):
#         tempkey= tempkey.decode("utf-8")
        #if isReadable(tempkey):
        #print(tempkey)
        #key=tempkey.strip()
        #print(i)


#     key_bv = [0]*BLOCKSIZE
#     key_bv = ''.join(str(j) for j in key_bv)


#     #print(key_bv)
#     for i in range(0,len(key) // numbytes): 
#         keyblock = key[i*numbytes:(i+1)*numbytes] 
#         keystr = textTobin(keyblock)
#         key_bv = bxor(key_bv,keystr)


    key_bv = st.zfill(BLOCKSIZE)
    msg = ""
    prev_block = binpass

    for i in range(0, len(content) // BLOCKSIZE): 
        bv = content[i*BLOCKSIZE:(i+1)*BLOCKSIZE]
        temp = bv

        bv = bxor(prev_block,bv)
        prev_block = temp

        bv = bxor(key_bv,bv)
        msg+= bv


    n = int(msg, 2)
    temp_n = len(hex(int(msg, 2))[2:])

    if(temp_n%2==1):
        msg = binascii.unhexlify('0%x' % n).decode("utf-8")
    else:
        msg = binascii.unhexlify('%x' % n).decode("utf-8")

    if isReadable(msg[:-1]):
        n = int(str(key_bv), 2)
        key = binascii.unhexlify('0%x' % n)
        print("KEY->",key,",","INDEX->",m)
        print(msg)

                

print("\n****KEY SEARCH FINISHED****")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe0 in position 1: invalid continuation byte